In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lasio
import xgboost as xgb
import scooby
import torch
import torch.nn as nn

from scipy.signal import filtfilt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import TensorDataset, DataLoader 
from torchsummary import summary
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error 
from sklearn.model_selection import train_test_split

# from dataset import *
# from model import *
# from train import *
# from utils import *

/home/taufikmh/miniconda3/envs/mlcoursegpu/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
# Load the training data
train_data = pd.read_csv('../../../labs/data/train.csv', sep=';')
train_data['WELL'] = train_data['WELL'].astype('category').cat.codes

# Load the testing data
test_data = pd.read_csv('../../../labs/data/leaderboard_test_features.csv', sep=';')
test_data['WELL'] = test_data['WELL'].astype('category').cat.codes

train_data = train_data[['WELL', 'DEPTH_MD', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTS', 'DTC']].copy().dropna()
test_data = test_data[~np.isnan(test_data.DTS.values)][['WELL', 'DEPTH_MD','CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTS', 'DTC']].copy().dropna()

train_data = train_data.mask(train_data>train_data.quantile(0.995), train_data.quantile(0.995), axis=1)
test_data = test_data.mask(test_data>test_data.quantile(0.995), test_data.quantile(0.995), axis=1)

# Select input
X_train = train_data[['CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC','WELL']].copy()
y_train = train_data[['DTS', 'WELL']].copy()

X_test = test_data[['CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC','WELL']].copy()
y_test = test_data[['DTS', 'WELL']].copy()

# Tranform RES logs
X_train['RSHA']=np.log(X_train.RSHA.values)
X_test['RSHA']=np.log(X_test.RSHA.values)
X_train['RMED']=np.log(X_train.RMED.values)
X_test['RMED']=np.log(X_test.RMED.values)
X_train['RDEP']=np.log(X_train.RDEP.values)
X_test['RDEP']=np.log(X_test.RDEP.values)

In [11]:
# Define Train Set
X_train = torch.from_numpy(X_train.to_numpy()).float().view(-1,X_train.shape[1])
y_train = torch.from_numpy(y_train.to_numpy()).float().view(-1,y_train.shape[1])
train_dataset = TensorDataset(X_train, y_train)

# Define Test Set
X_test = torch.from_numpy(X_test.to_numpy()).float().view(-1,X_test.shape[1])
y_test = torch.from_numpy(y_test.to_numpy()).float().view(-1,y_test.shape[1])
valid_dataset = TensorDataset(X_test, y_test)

# Use Pytorch's functionality to load data in batches. Here we use full-batch training again.
train_loader = DataLoader(train_dataset, batch_size=X_train.size(0), shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=X_test.size(0), shuffle=False)

torch.save(train_loader, '../data/train_loader.pth')    
torch.save(valid_loader, '../data/valid_loader.pth')   